# Project 2 - Couch db 
* Group 2

# Installation

* WSL / Linux:      
    There are several ways to download on WSL/Linux, from source, docker and the snap store. I found the easiest way to download the file was the snap store, I had several errors with the docker version and the cource would not install the couchdb package.

    * Command 1:  
    sudo snap install couchdb

    * Command 2:    
    sudo snap set couchdb admin=[*password] setcookie=[]
    
    This will set your login information and a cookie for session information as it is web based.

    * command 2 example:
    sudo snap set couchdb admin=[abc] setcookie=[random]

    This is an example of the command filled in, random is an option for the cookie argument. 


* Windows

    select windows box on installations, click download, launch file named "apache-couchdb-version". Fill out prompts for username and password, validate credentials, do the same steps for the cookie.

***

## Verify installation for Linux

  First, check if you even have couchdb as a snap. You can do this through a simple command.

* Command 1:

  snap list

  should show something like: couchdb      3.4.2             87     latest/stable    apache-couchdb  -


   Next, start the service. Snap can be a bit ficky so you'll have to do this command:

  * Command 2

   sudo snap start couchdb 

    Finally, open web browser to http://localhost:5984/_utils (ports defined when installing) and it will bring you to a login page, enter your credentails and you will have access to the dashboard.


  If for whatever reason, the username admin with the password you set doesn't work at all, you'll have to edit this in the local.ini file with a plain text password. For the directory path, it should be at /var/snap/couchdb/87/etc/local.ini. Do the following:

  
* sudo nano /var/snap/couchdb/87/etc/local.ini

   You'll see in the section [admins] that admin might have a hashed value. Here's what it has:
   
 To create an admin account uncomment the '[admins]' section below and add a
 line in the format 'username = password'. When you next start CouchDB, it
 will change the password to a hash (so that your passwords don't linger
 around in plain-text files). You can add more admin accounts with more
 'username = password' lines. Don't forget to restart CouchDB after
 changing this.

[admins]
admin = -pbkdf2:sha256-...

   
   Remove that and put your password you set from  sudo snap set couchdb admin=[*password] setcookie=[]

In my case, I did admin = abc, then I did ctrl o, enter, then ctrl x to exit. Finally, you should reset the couchdb so it saves that password with this one command to restart the service:

 sudo snap restart couchdb
***

## Verify installation for Windows

  This should work out of the box. Open web browser to http://localhost:5984/_utils (ports defined when installing) and it will bring you to a login page, enter your credentails and you will have access to the dashboard.


# Requirements

* Apache CouchDB uses a web based interface to interact with the service, a browser is required. 

* No offical hardware Requirements for running couchdb, besides a network connection. 

* Runs on most hardware, even forum posts of couchdb running on 1 core, 512 Mb of ram and 20Gb of storage. Extremely flexible DB system. 

    # Support

    * Has support for popular linux distros such as CentOS, Debian, and Ubuntu; Support for FreeBSD, MacOS, and Windows. 

    * if the device can run the Snap store or Docker couchdb can be downloaded and initialized.


***
## This section will be on Databases from Creating, Deleting, and Retriving them all within CouchDB
***

### Running the DB to Authenicate with your user and create Database

In [6]:
import requests
from requests.auth import HTTPBasicAuth

# Set up the CouchDB URL
couch_url = "http://127.0.0.1:5984/"

# Correct admin username and password
username = "admin"  # Your CouchDB admin username
password = "abc"  # Your actual CouchDB admin password

# Create a database in CouchDB (Be sure to change within the quotes of the name you desire)
response = requests.put(couch_url + "my_database2", auth=HTTPBasicAuth(username, password))

# Print out the response
print(response.status_code)  # Check if it was successful (should be 201 for created) 
                             # (Otherwise error 412 as it's the same DB you made)
print(response.json())  # Detailed response from CouchDB


### Deletes Database within couchDB. Change the db you have in the quotes for which one you choose to delete

In [ ]:
response = requests.delete(couch_url + "my_database", auth=HTTPBasicAuth(username, password))
print(response.status_code)  # Should return 200 if the database was deleted
                             # Otherwise, error 404 for the database that doesn't exist
print(response.json())  # More info about the deletion


### Retreives all Databases stored within CouchDB. DO NOT change "_all_dbs", that is there on purpose for the requests library to function

In [ ]:
response = requests.get(couch_url + "_all_dbs", auth=HTTPBasicAuth(username, password))
print(response.status_code)
print(response.json())  # List of all databases you created.


***
## This section will be on interacting with the documents themselves to parse JSON data within the Databases in CouchDB
***

### Creating Documents in your desired DB

In [ ]:
# Define the document, this is through JSON as this is a document based DB.
document = {
    "name": "John Doe",
    "email": "johndoe@example.com",
    "age": 30
}

# Insert the document into the DB (be sure to change DB into what you want)
response = requests.post(couch_url + "my_database", json=document, auth=HTTPBasicAuth(username, password))
print(response.status_code)  # Should return 201 for created
print(response.json())  # Document details, including the _id and _rev


### Outputs the Document JSON | Should change doc_id and whatever your db is in response

In [ ]:
doc_id = "Some_document_ID"  # Replace with your actual document ID
response = requests.get(couch_url + "my_database/" + doc_id, auth=HTTPBasicAuth(username, password))
print(response.status_code)
print(response.json())  # The content of the document


### Modification of the document |  Change doc_id and whatever db is in response

In [ ]:
# Example: Retrieve the document, modify it, and send it back with _rev
doc_id = "Some_document_ID"  # Replace with your actual document ID
response = requests.get(couch_url + "my_database/" + doc_id, auth=HTTPBasicAuth(username, password))

if response.status_code == 200:
    document = response.json()
    document["age"] = 31  # Update the document

    # Send the updated document
    response = requests.put(couch_url + "my_database/" + doc_id, json=document, auth=HTTPBasicAuth(username, password))
    print(response.status_code)  # Should return 201 if the update is successful
    print(response.json())  # Updated document details


### Deletion of document | Change doc_id and whatever db is in response

In [ ]:
doc_id = "Some_document_ID"  # Replace with your actual document ID
response = requests.get(couch_url + "my_database/" + doc_id, auth=HTTPBasicAuth(username, password))

if response.status_code == 200:
    document = response.json()
    rev = document["_rev"]  # Get the revision ID for deletion

    # Delete the document
    response = requests.delete(couch_url + "my_database/" + doc_id + "?rev=" + rev, auth=HTTPBasicAuth(username, password))
    print(response.status_code)  # Should return 200 for successful deletion
    print(response.json())  # Confirmation of deletion
